In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.ensemble import RandomForestRegressor
from hdbscan import HDBSCAN
import umap
from datetime import datetime, timedelta
import os
from tqdm import tqdm
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap
import warnings

c:\Users\Kshitij\OneDrive\Desktop\RozReturns\market_regime\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

In [8]:
def load_depth_data(file_path):
    """Load order‑book depth data from file, parsing the IST timestamps."""
    print(f"Loading depth data from {file_path}...")
    df = pd.read_csv(file_path)
    
    # 1) Remove the trailing " IST"
    # 2) Parse with pandas (it handles the "+0530" offset and nanoseconds)
    df['Time'] = (
        df['Time']
          .str.replace(r'\s+IST$', '', regex=True)
          .pipe(pd.to_datetime, utc=True)   # becomes UTC; you can .dt.tz_convert('Asia/Kolkata') if you want local time
    )
    df.set_index('Time', inplace=True)
    return df

def load_trade_data(file_path):
    """Load trade data from file, parsing the IST timestamps."""
    print(f"Loading trade data from {file_path}...")
    df = pd.read_csv(file_path)
    df['Time'] = (
        df['Time']
          .str.replace(r'\s+IST$', '', regex=True)
          .pipe(pd.to_datetime, utc=True)
    )
    df.set_index('Time', inplace=True)
    return df

def load_all_data(base_path, date_range):
    """Load all depth and trade data for given date range."""
    depth_data = {}
    trade_data = {}
    
    for date in date_range:
        date_str = date.strftime('%Y%m%d')
        depth_file = os.path.join(base_path, 'depth20_1000ms', f'BNBFDUSD_{date_str}.txt')
        trade_file = os.path.join(base_path, 'aggTrade',      f'BNBFDUSD_{date_str}.txt')
        
        if os.path.exists(depth_file):
            depth_data[date_str] = load_depth_data(depth_file)
        
        if os.path.exists(trade_file):
            trade_data[date_str] = load_trade_data(trade_file)
    
    return depth_data, trade_data

In [9]:
# Define date range for analysis (March 14-17, 2025)
date_range = [datetime(2025, 3, 14) + timedelta(days=i) for i in range(4)]

# Update with your actual data path
base_path = "./data"

In [10]:
depth_data, trade_data = load_all_data(base_path, date_range)

Loading depth data from ./data\depth20_1000ms\BNBFDUSD_20250314.txt...
Loading trade data from ./data\aggTrade\BNBFDUSD_20250314.txt...
Loading depth data from ./data\depth20_1000ms\BNBFDUSD_20250315.txt...
Loading trade data from ./data\aggTrade\BNBFDUSD_20250315.txt...
Loading depth data from ./data\depth20_1000ms\BNBFDUSD_20250316.txt...
Loading trade data from ./data\aggTrade\BNBFDUSD_20250316.txt...
Loading depth data from ./data\depth20_1000ms\BNBFDUSD_20250317.txt...
Loading trade data from ./data\aggTrade\BNBFDUSD_20250317.txt...


In [11]:
def create_synthetic_data():
    """Create synthetic data for demonstration."""
    print("Creating synthetic data for demonstration...")
    
    # Create timestamps for 4 days, every second
    start_date = datetime(2025, 3, 14)
    dates = {}
    trade_dates = {}
    
    for i in range(4):
        date = start_date + timedelta(days=i)
        date_str = date.strftime('%Y%m%d')
        
        # Create timestamps for depth data (every second)
        timestamps = [date + timedelta(seconds=j) for j in range(86400)]  # 24 hours
        
        # Create synthetic depth data
        depth_df = pd.DataFrame(index=timestamps)
        
        # Create bid columns
        for level in range(1, 21):
            base_price = 605.0 + np.random.normal(0, 0.5)
            depth_df[f'BidPriceL{level}'] = [base_price - 0.01*level + np.random.normal(0, 0.02) for _ in range(len(timestamps))]
            depth_df[f'BidQtyL{level}'] = [np.random.exponential(2) for _ in range(len(timestamps))]
        
        # Create ask columns
        for level in range(1, 21):
            base_price = 605.2 + np.random.normal(0, 0.5)
            depth_df[f'AskPriceL{level}'] = [base_price + 0.01*level + np.random.normal(0, 0.02) for _ in range(len(timestamps))]
            depth_df[f'AskQtyL{level}'] = [np.random.exponential(2) for _ in range(len(timestamps))]
        
        dates[date_str] = depth_df
        
        # Create synthetic trade data (fewer trades than depth data)
        trade_timestamps = sorted(np.random.choice(timestamps, size=int(len(timestamps)*0.05), replace=False))
        trade_df = pd.DataFrame(index=trade_timestamps)
        trade_df['Price'] = [605.0 + np.random.normal(0, 0.2) for _ in range(len(trade_timestamps))]
        trade_df['Quantity'] = [np.random.exponential(1) for _ in range(len(trade_timestamps))]
        trade_df['IsMarketMaker'] = [np.random.choice([True, False], p=[0.3, 0.7]) for _ in range(len(trade_timestamps))]
        trade_df['NumTrades'] = [np.random.randint(1, 5) for _ in range(len(trade_timestamps))]
        trade_df['M'] = True  # Assuming 'M' is some flag always set to True
        
        trade_dates[date_str] = trade_df
    
    return dates, trade_dates

depth_data, trade_data = create_synthetic_data()

Creating synthetic data for demonstration...


In [12]:
# 2. Feature Engineering

def extract_basic_features(depth_df, window_sizes=[10, 30, 60]):
    """Extract basic features from depth data."""
    print("Extracting basic features from depth data...")
    
    features = pd.DataFrame(index=depth_df.index)
    
    # Liquidity & Depth Features
    features['bid_ask_spread'] = depth_df['AskPriceL1'] - depth_df['BidPriceL1']
    features['mid_price'] = (depth_df['AskPriceL1'] + depth_df['BidPriceL1']) / 2
    
    # Order book imbalance at each level
    for level in range(1, 6):  # Computing for first 5 levels
        bid_qty = depth_df[f'BidQtyL{level}']
        ask_qty = depth_df[f'AskQtyL{level}']
        features[f'imbalance_lvl{level}'] = (bid_qty - ask_qty) / (bid_qty + ask_qty + 1e-10)
    
    # Microprice
    features['microprice'] = (depth_df['BidPriceL1'] * depth_df['AskQtyL1'] + 
                             depth_df['AskPriceL1'] * depth_df['BidQtyL1']) / (
                             depth_df['BidQtyL1'] + depth_df['AskQtyL1'] + 1e-10)
    
    # Cumulative depth
    cum_bid_qty = sum(depth_df[f'BidQtyL{level}'] for level in range(1, 21))
    cum_ask_qty = sum(depth_df[f'AskQtyL{level}'] for level in range(1, 21))
    features['cum_bid_qty'] = cum_bid_qty
    features['cum_ask_qty'] = cum_ask_qty
    features['cum_qty_imbalance'] = (cum_bid_qty - cum_ask_qty) / (cum_bid_qty + cum_ask_qty + 1e-10)
    
    # Sloped depth calculation
    bid_slope = []
    ask_slope = []
    
    for i in range(len(depth_df)):
        # Calculate average slope in bid side (how quickly quantity decreases)
        bid_prices = [depth_df[f'BidPriceL{level}'].iloc[i] for level in range(1, 21)]
        bid_qtys = [depth_df[f'BidQtyL{level}'].iloc[i] for level in range(1, 21)]
        bid_vwap = sum(p*q for p, q in zip(bid_prices, bid_qtys)) / (sum(bid_qtys) + 1e-10)
        bid_slope.append(abs(bid_vwap - bid_prices[0]))
        
        # Calculate average slope in ask side
        ask_prices = [depth_df[f'AskPriceL{level}'].iloc[i] for level in range(1, 21)]
        ask_qtys = [depth_df[f'AskQtyL{level}'].iloc[i] for level in range(1, 21)]
        ask_vwap = sum(p*q for p, q in zip(ask_prices, ask_qtys)) / (sum(ask_qtys) + 1e-10)
        ask_slope.append(abs(ask_vwap - ask_prices[0]))
    
    features['bid_slope'] = bid_slope
    features['ask_slope'] = ask_slope
    
    # Volatility & Price Action Features
    features['mid_price_log_return'] = np.log(features['mid_price'] / features['mid_price'].shift(1)).fillna(0)
    
    # Calculate rolling statistics over multiple windows
    for window in window_sizes:
        features[f'volatility_{window}s'] = features['mid_price_log_return'].rolling(window=window).std().fillna(0)
        features[f'trend_{window}s'] = features['mid_price_log_return'].rolling(window=window).mean().fillna(0)
        
        # Price acceleration (second derivative)
        features[f'price_acceleration_{window}s'] = features['mid_price_log_return'].diff().rolling(window=window).mean().fillna(0)
        
        # Liquidity acceleration 
        features[f'spread_acceleration_{window}s'] = features['bid_ask_spread'].diff().rolling(window=window).mean().fillna(0)
    
    # Add cyclical time features
    features['hour_sin'] = np.sin(2 * np.pi * features.index.hour / 24)
    features['hour_cos'] = np.cos(2 * np.pi * features.index.hour / 24)
    features['day_of_week'] = features.index.dayofweek
    features['day_sin'] = np.sin(2 * np.pi * features.index.dayofweek / 7)
    features['day_cos'] = np.cos(2 * np.pi * features.index.dayofweek / 7)
    
    return features

def add_trade_features(features_df, trade_df, window_sizes=[10, 30, 60]):
    """Add trade features to existing feature dataframe."""
    print("Adding trade features to the dataset...")
    
    # Resample trade data to match feature index
    trade_resampled = pd.DataFrame(index=features_df.index)
    
    # Prepare trade volume data
    trade_df['is_buy'] = trade_df['Price'] >= trade_df['Price'].shift(1)
    trade_df['buy_volume'] = np.where(trade_df['is_buy'], trade_df['Quantity'], 0)
    trade_df['sell_volume'] = np.where(~trade_df['is_buy'], trade_df['Quantity'], 0)
    
    # Resample trade data to feature index
    for window in window_sizes:
        window_seconds = f'{window}s'
        
        # Calculate trade volumes in each window
        trade_vol = trade_df['Quantity'].resample(window_seconds).sum().reindex(features_df.index, method='ffill').fillna(0)
        buy_vol = trade_df['buy_volume'].resample(window_seconds).sum().reindex(features_df.index, method='ffill').fillna(0)
        sell_vol = trade_df['sell_volume'].resample(window_seconds).sum().reindex(features_df.index, method='ffill').fillna(0)
        
        # Add trade features to main features dataframe
        features_df[f'trade_volume_{window}s'] = trade_vol
        features_df[f'buy_volume_{window}s'] = buy_vol
        features_df[f'sell_volume_{window}s'] = sell_vol
        features_df[f'volume_imbalance_{window}s'] = (buy_vol - sell_vol) / (buy_vol + sell_vol + 1e-10)
        
        # Count number of trades in window
        trade_count = trade_df['NumTrades'].resample(window_seconds).count().reindex(features_df.index, method='ffill').fillna(0)
        features_df[f'trade_count_{window}s'] = trade_count
        
        # Calculate VWAP
        vwap_num = (trade_df['Price'] * trade_df['Quantity']).resample(window_seconds).sum().reindex(features_df.index, method='ffill').fillna(0)
        vwap_den = trade_df['Quantity'].resample(window_seconds).sum().reindex(features_df.index, method='ffill').fillna(0)
        features_df[f'vwap_{window}s'] = vwap_num / (vwap_den + 1e-10)
        
        # Calculate VWAP shift
        features_df[f'vwap_shift_{window}s'] = features_df[f'vwap_{window}s'].diff().fillna(0)
        
        # Volume acceleration (second derivative)
        features_df[f'volume_acceleration_{window}s'] = features_df[f'trade_volume_{window}s'].diff().rolling(window).mean().fillna(0)
    
    # Market maker participation ratio
    mm_volume = trade_df[trade_df['IsMarketMaker']]['Quantity'].resample('60s').sum().reindex(features_df.index, method='ffill').fillna(0)
    total_volume = trade_df['Quantity'].resample('60s').sum().reindex(features_df.index, method='ffill').fillna(0)
    features_df['mm_participation_ratio'] = mm_volume / (total_volume + 1e-10)
    
    return features_df

# Process one day of data as an example
date_key = list(depth_data.keys())[0]
depth_df = depth_data[date_key]
trade_df = trade_data[date_key]

# Extract features
features = extract_basic_features(depth_df)
features = add_trade_features(features, trade_df)

# Clean any potential NaN values
features.fillna(0, inplace=True)

# Display sample of the features
print("\nFeature dataset sample:")
print(features.head())
print(f"\nTotal features: {features.shape[1]}")

Extracting basic features from depth data...
Adding trade features to the dataset...

Feature dataset sample:
                     bid_ask_spread   mid_price  imbalance_lvl1  \
2025-03-14 00:00:00        1.291286  604.676983       -0.051213   
2025-03-14 00:00:01        1.345523  604.658099        0.762085   
2025-03-14 00:00:02        1.307937  604.678843       -0.669070   
2025-03-14 00:00:03        1.292335  604.669307       -0.456128   
2025-03-14 00:00:04        1.299332  604.673042        0.002897   

                     imbalance_lvl2  imbalance_lvl3  imbalance_lvl4  \
2025-03-14 00:00:00        0.719433       -0.700780        0.028288   
2025-03-14 00:00:01       -0.421668       -0.376748        0.310946   
2025-03-14 00:00:02        0.893577       -0.179744        0.162550   
2025-03-14 00:00:03        0.520779       -0.763065        0.471272   
2025-03-14 00:00:04       -0.967221        0.304086        0.019762   

                     imbalance_lvl5  microprice  cum_bid_qty

In [13]:
# 3. Data Normalization and Dimensionality Reduction

def normalize_and_reduce_dimensions(features_df, n_components=20):
    """Normalize features and reduce dimensionality using PCA."""
    print("\nNormalizing features and reducing dimensions...")
    
    # Remove non-numeric columns
    numeric_features = features_df.select_dtypes(include=[np.number])
    
    # Normalize using StandardScaler
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(numeric_features)
    
    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=n_components)
    features_reduced = pca.fit_transform(features_scaled)
    
    # Create dataframe with reduced dimensions
    reduced_df = pd.DataFrame(features_reduced, 
                              index=numeric_features.index,
                              columns=[f'PC{i+1}' for i in range(n_components)])
    
    # Display explained variance ratio
    explained_variance = pca.explained_variance_ratio_
    cumulative_variance = np.cumsum(explained_variance)
    
    print(f"Explained variance by {n_components} principal components: {cumulative_variance[-1]:.2%}")
    
    # Feature importance
    feature_importance = pd.DataFrame(
        pca.components_.T,
        columns=[f'PC{i+1}' for i in range(n_components)],
        index=numeric_features.columns
    )
    
    return reduced_df, scaler, pca, feature_importance

# Apply normalization and dimensionality reduction
reduced_features, scaler, pca, feature_importance = normalize_and_reduce_dimensions(features)

# Display reduced features
print("\nReduced features sample:")
print(reduced_features.head())

# Plot explained variance
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_)
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), np.cumsum(pca.explained_variance_ratio_), 'r-')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance Ratio')
plt.title('Explained Variance by Principal Components')
plt.grid(True)
plt.savefig('pca_explained_variance.png', dpi=300, bbox_inches='tight')
plt.close()



Normalizing features and reducing dimensions...
Explained variance by 20 principal components: 77.18%

Reduced features sample:
                          PC1       PC2       PC3       PC4       PC5  \
2025-03-14 00:00:00 -1.357905  0.258623  2.381864 -0.120988 -0.413091   
2025-03-14 00:00:01 -1.361249 -0.840869  2.412753  0.613030 -0.448101   
2025-03-14 00:00:02 -1.354613  0.648232  2.400058  0.458326 -0.447854   
2025-03-14 00:00:03 -1.363294 -0.335481  2.388326 -0.022886 -0.400603   
2025-03-14 00:00:04 -1.348784  0.413845  2.340187 -0.169862 -0.267976   

                          PC6        PC7       PC8       PC9      PC10  \
2025-03-14 00:00:00  0.055259  10.675446  0.384726 -0.624912 -0.166244   
2025-03-14 00:00:01  1.467882  10.679077 -1.144993 -0.549370  0.038826   
2025-03-14 00:00:02  0.003800  10.745238  2.779333 -0.709034 -0.182637   
2025-03-14 00:00:03 -0.587467  10.683595  1.448986 -0.631417 -0.068137   
2025-03-14 00:00:04 -1.925196  10.541746 -2.408695 -0.508975 -

In [14]:
# 4. Ensemble Clustering

def apply_kmeans(data, k_range=range(2, 11)):
    """Apply K-means clustering with various K values and evaluate."""
    print("\nApplying K-means clustering...")
    
    results = []
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(data)
        
        silhouette = silhouette_score(data, labels) if len(set(labels)) > 1 else -1
        db_index = davies_bouldin_score(data, labels) if len(set(labels)) > 1 else float('inf')
        
        results.append({
            'k': k,
            'silhouette': silhouette,
            'davies_bouldin': db_index,
            'labels': labels,
            'model': kmeans
        })
    
    # Find best model based on silhouette score
    best_result = max(results, key=lambda x: x['silhouette'])
    
    print(f"Best K-means model: k={best_result['k']}, silhouette={best_result['silhouette']:.4f}")
    
    return results, best_result

def apply_gmm(data, k_range=range(2, 11)):
    """Apply Gaussian Mixture Model clustering with various K values and evaluate."""
    print("\nApplying Gaussian Mixture Model clustering...")
    
    results = []
    
    for k in k_range:
        gmm = GaussianMixture(n_components=k, random_state=42, n_init=10)
        gmm.fit(data)
        labels = gmm.predict(data)
        
        silhouette = silhouette_score(data, labels) if len(set(labels)) > 1 else -1
        db_index = davies_bouldin_score(data, labels) if len(set(labels)) > 1 else float('inf')
        
        results.append({
            'k': k,
            'silhouette': silhouette,
            'davies_bouldin': db_index,
            'labels': labels,
            'model': gmm
        })
    
    # Find best model based on silhouette score
    best_result = max(results, key=lambda x: x['silhouette'])
    
    print(f"Best GMM model: k={best_result['k']}, silhouette={best_result['silhouette']:.4f}")
    
    return results, best_result

In [15]:
def apply_hdbscan(data, min_samples_range=[5, 10, 15, 20, 30], min_cluster_size_range=[10, 20, 30, 50, 100]):
    """Apply HDBSCAN clustering with various parameter values and evaluate."""
    print("\nApplying HDBSCAN clustering...")
    
    results = []
    
    for min_samples in min_samples_range:
        for min_cluster_size in min_cluster_size_range:
            try:
                hdbscan = HDBSCAN(min_samples=min_samples, min_cluster_size=min_cluster_size)
                labels = hdbscan.fit_predict(data)
                
                # Skip if all points are classified as noise (-1)
                if len(set(labels)) <= 1 or all(l == -1 for l in labels):
                    continue
                
                # Replace noise points (-1) temporarily for metric calculation
                temp_labels = np.array(labels)
                if -1 in temp_labels:
                    # Assign noise points to nearest cluster for metric calculation
                    noise_indices = np.where(temp_labels == -1)[0]
                    non_noise_indices = np.where(temp_labels != -1)[0]
                    
                    if len(non_noise_indices) > 0:
                        from sklearn.neighbors import NearestNeighbors
                        nn = NearestNeighbors(n_neighbors=1)
                        nn.fit(data[non_noise_indices])
                        _, indices = nn.kneighbors(data[noise_indices])
                        
                        for i, idx in enumerate(noise_indices):
                            temp_labels[idx] = temp_labels[non_noise_indices[indices[i][0]]]
                
                # Calculate metrics if we have more than one cluster
                if len(set(temp_labels)) > 1:
                    silhouette = silhouette_score(data, temp_labels)
                    db_index = davies_bouldin_score(data, temp_labels)
                else:
                    silhouette = -1
                    db_index = float('inf')
                
                results.append({
                    'min_samples': min_samples,
                    'min_cluster_size': min_cluster_size,
                    'silhouette': silhouette,
                    'davies_bouldin': db_index,
                    'labels': labels,
                    'model': hdbscan
                })
            except Exception as e:
                print(f"Error with HDBSCAN parameters min_samples={min_samples}, min_cluster_size={min_cluster_size}: {e}")
    
    if not results:
        print("No valid HDBSCAN results found. Using default parameters.")
        hdbscan = HDBSCAN(min_samples=5, min_cluster_size=20)
        labels = hdbscan.fit_predict(data)
        results.append({
            'min_samples': 5,
            'min_cluster_size': 20,
            'silhouette': -1,
            'davies_bouldin': float('inf'),
            'labels': labels,
            'model': hdbscan
        })
    
    # Find best model based on silhouette score
    best_result = max(results, key=lambda x: x['silhouette'])
    
    print(f"Best HDBSCAN model: min_samples={best_result['min_samples']}, min_cluster_size={best_result['min_cluster_size']}, silhouette={best_result['silhouette']:.4f}")
    
    return results, best_result

In [16]:
def create_ensemble_clustering(kmeans_labels, gmm_labels, hdbscan_labels, n_clusters=None):
    """Create ensemble clustering by combining results from multiple algorithms."""
    print("\nCreating ensemble clustering...")
    
    # Stack labels from different algorithms
    combined_data = np.column_stack([kmeans_labels, gmm_labels, hdbscan_labels])
    
    # Apply K-means on the combined labels to get consensus clusters
    if n_clusters is None:
        n_clusters = len(set(kmeans_labels))  # Use number of clusters from K-means as default
    
    consensus_kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    consensus_labels = consensus_kmeans.fit_predict(combined_data)
    
    # Evaluate ensemble results
    silhouette = silhouette_score(reduced_features, consensus_labels)
    db_index = davies_bouldin_score(reduced_features, consensus_labels)
    
    print(f"Ensemble clustering: n_clusters={n_clusters}, silhouette={silhouette:.4f}, davies_bouldin={db_index:.4f}")
    
    return consensus_labels, consensus_kmeans

In [17]:
# Apply different clustering algorithms
kmeans_results, best_kmeans = apply_kmeans(reduced_features)
gmm_results, best_gmm = apply_gmm(reduced_features)
hdbscan_results, best_hdbscan = apply_hdbscan(reduced_features)


Applying K-means clustering...
Best K-means model: k=2, silhouette=0.1424

Applying Gaussian Mixture Model clustering...
Best GMM model: k=2, silhouette=0.0729

Applying HDBSCAN clustering...
Error with HDBSCAN parameters min_samples=5, min_cluster_size=10: "None of [Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,\n       ...\n       86390, 86391, 86392, 86393, 86394, 86395, 86396, 86397, 86398, 86399],\n      dtype='int64', length=86321)] are in the [columns]"
Error with HDBSCAN parameters min_samples=5, min_cluster_size=100: "None of [Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,\n       ...\n       86390, 86391, 86392, 86393, 86394, 86395, 86396, 86397, 86398, 86399],\n      dtype='int64', length=86156)] are in the [columns]"
Error with HDBSCAN parameters min_samples=10, min_cluster_size=100: "None of [Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,\n       ...\n       86390, 86391, 86392, 8639

In [19]:
# Create ensemble clustering
ensemble_labels, ensemble_model = create_ensemble_clustering(
    best_kmeans['labels'], 
    best_gmm['labels'], 
    best_hdbscan['labels']
)

# Plot silhouette scores for different K values
plt.figure(figsize=(12, 6))
plt.plot([r['k'] for r in kmeans_results], [r['silhouette'] for r in kmeans_results], 'o-', label='K-means')
plt.plot([r['k'] for r in gmm_results], [r['silhouette'] for r in gmm_results], 's-', label='GMM')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Scores for Different Clustering Methods')
plt.legend()
plt.grid(True)
plt.savefig('silhouette_scores.png', dpi=300, bbox_inches='tight')
plt.close()


Creating ensemble clustering...
Ensemble clustering: n_clusters=2, silhouette=0.1424, davies_bouldin=2.6031


In [20]:
# 5. Regime Labeling and Analysis

def analyze_clusters(features_df, reduced_df, labels, analysis_features=None):
    """Analyze characteristics of each cluster/regime."""
    print("\nAnalyzing cluster characteristics...")
    
    if analysis_features is None:
        # Select most important features for analysis
        analysis_features = [
            'bid_ask_spread', 'mid_price_log_return', 'volatility_30s', 'trend_30s',
            'cum_qty_imbalance', 'volume_imbalance_30s', 'trade_volume_30s',
            'price_acceleration_30s', 'volume_acceleration_30s'
        ]
    
    # Add labels to original features dataframe
    labeled_df = features_df.copy()
    labeled_df['cluster'] = labels
    
    # Calculate statistics for each cluster
    cluster_stats = labeled_df.groupby('cluster')[analysis_features].mean()
    
    # Calculate standard deviations for each cluster
    cluster_stds = labeled_df.groupby('cluster')[analysis_features].std()
    
    # Determine regime characteristics
    regime_characteristics = {}
    
    for cluster in cluster_stats.index:
        stats = cluster_stats.loc[cluster]
        
        # Trending vs Mean-reverting
        trending_score = stats['trend_30s'] / (cluster_stats['trend_30s'].std() + 1e-10)
        regime_trend = "Trending" if abs(trending_score) > 0.5 else "Mean-Reverting"
        if trending_score > 0.5:
            regime_trend = "Trending Up"
        elif trending_score < -0.5:
            regime_trend = "Trending Down"
        
        # Volatile vs Stable
        volatility_score = stats['volatility_30s'] / (cluster_stats['volatility_30s'].mean() + 1e-10)
        regime_volatility = "Volatile" if volatility_score > 1.2 else "Stable"
        
        # Liquid vs Illiquid
        liquidity_score = stats['bid_ask_spread'] / (cluster_stats['bid_ask_spread'].mean() + 1e-10)
        regime_liquidity = "Illiquid" if liquidity_score > 1.2 else "Liquid"
        
        # Create regime name
        regime_name = f"{regime_trend} & {regime_liquidity} & {regime_volatility}"
        
        # Additional insights
        imbalance = "Buy Pressure" if stats['cum_qty_imbalance'] > 0.1 else "Sell Pressure" if stats['cum_qty_imbalance'] < -0.1 else "Balanced"
        
        regime_characteristics[cluster] = {
            'name': regime_name,
            'trend': regime_trend,
            'volatility': regime_volatility,
            'liquidity': regime_liquidity,
            'imbalance': imbalance,
            'stats': stats
        }
    
    return regime_characteristics, labeled_df

# Analyze ensemble clustering results
regime_characteristics, labeled_df = analyze_clusters(features, reduced_features, ensemble_labels)

# Display regime characteristics
print("\nRegime Characteristics:")
for cluster, details in regime_characteristics.items():
    print(f"Regime {cluster} ({details['name']}):")
    print(f"  - Trend: {details['trend']}")
    print(f"  - Volatility: {details['volatility']}")
    print(f"  - Liquidity: {details['liquidity']}")
    print(f"  - Order Book Pressure: {details['imbalance']}")
    print()



Analyzing cluster characteristics...

Regime Characteristics:
Regime 0 (Trending Down & Liquid & Stable):
  - Trend: Trending Down
  - Volatility: Stable
  - Liquidity: Liquid
  - Order Book Pressure: Balanced

Regime 1 (Mean-Reverting & Liquid & Stable):
  - Trend: Mean-Reverting
  - Volatility: Stable
  - Liquidity: Liquid
  - Order Book Pressure: Balanced



In [21]:
# 6. Market Impact Analysis

def analyze_market_impact(labeled_df, trade_df):
    """Analyze how trades impact the market in different regimes."""
    print("\nAnalyzing market impact in different regimes...")
    
    # Create a dataframe to store market impact results
    impact_results = pd.DataFrame(columns=['regime', 'trade_size_quantile', 'price_impact', 'recovery_time'])
    
    # Merge trade data with regime labels
    labeled_trades = pd.DataFrame(index=trade_df.index)
    labeled_trades['Price'] = trade_df['Price']
    labeled_trades['Quantity'] = trade_df['Quantity']
    
    # Assign regime to each trade (nearest in time)
    labeled_trade_indices = labeled_trades.index
    nearest_indices = []
    
    for trade_time in labeled_trade_indices:
        # Find the nearest timestamp in labeled_df
        nearest_idx = labeled_df.index[labeled_df.index.get_indexer([trade_time], method='nearest')[0]]
        nearest_indices.append(nearest_idx)
    
    labeled_trades['regime'] = [labeled_df.loc[idx, 'cluster'] for idx in nearest_indices]
    
    # Analyze price impact for different trade size quantiles in each regime
    regimes = labeled_trades['regime'].unique()
    quantiles = [0.25, 0.5, 0.75, 0.9]  # Small, medium, large, very large trades

    for regime in regimes:
        regime_trades = labeled_trades[labeled_trades['regime'] == regime]
        
        # Skip if not enough trades
        if len(regime_trades) < 20:
            continue
        
        # Calculate trade size quantiles
        size_quantiles = regime_trades['Quantity'].quantile(quantiles)
        
        for i, q in enumerate(quantiles):
            quantile_name = f"Q{int(q*100)}"
            quantile_value = size_quantiles[q]
            
            # Filter trades in this size quantile
            if i < len(quantiles) - 1:
                size_mask = (regime_trades['Quantity'] >= quantile_value) & (regime_trades['Quantity'] < size_quantiles[quantiles[i+1]])
            else:
                size_mask = (regime_trades['Quantity'] >= quantile_value)
            
            quantile_trades = regime_trades[size_mask]
            
            # Skip if not enough trades in this quantile
            if len(quantile_trades) < 10:
                continue
            
            # Calculate average price impact (5 seconds after trade)
            avg_impact = 0
            avg_recovery = 0
            
            for idx, trade in quantile_trades.iterrows():
                # Find price before and after trade
                try:
                    # Price before trade
                    pre_trade_time = idx - timedelta(seconds=1)
                    pre_trade_price = labeled_df.loc[labeled_df.index.get_indexer([pre_trade_time], method='nearest')[0], 'mid_price']
                    
                    # Price immediately after trade (impact)
                    post_trade_time = idx + timedelta(seconds=5)
                    post_idx = labeled_df.index.get_indexer([post_trade_time], method='nearest')[0]
                    post_trade_price = labeled_df.iloc[post_idx]['mid_price']
                    
                    # Price recovery (30 seconds after trade)
                    recovery_time = idx + timedelta(seconds=30)
                    recovery_idx = labeled_df.index.get_indexer([recovery_time], method='nearest')[0]
                    recovery_price = labeled_df.iloc[recovery_idx]['mid_price']
                    
                    # Calculate impact and recovery
                    impact = (post_trade_price - pre_trade_price) / pre_trade_price
                    recovery = 1 - abs((recovery_price - pre_trade_price) / (post_trade_price - pre_trade_price + 1e-10))
                    
                    avg_impact += impact
                    avg_recovery += recovery
                except:
                    # Skip if can't calculate impact
                    continue
            
            if len(quantile_trades) > 0:
                avg_impact /= len(quantile_trades)
                avg_recovery /= len(quantile_trades)
                
                # Add to results
                impact_results = impact_results.append({
                    'regime': regime_characteristics[regime]['name'],
                    'trade_size_quantile': quantile_name,
                    'price_impact': avg_impact * 10000,  # Convert to basis points
                    'recovery_time': avg_recovery
                }, ignore_index=True)
    
    return impact_results

In [22]:
# Run market impact analysis
# impact_results = analyze_market_impact(labeled_df, trade_data[list(trade_data.keys())[0]])
# With simulated data we'll create a synthetic impact analysis
def create_synthetic_impact_analysis():
    """Create synthetic market impact analysis results for demonstration."""
    impact_results = []
    
    for regime_id, regime_info in regime_characteristics.items():
        regime_name = regime_info['name']
        
        # Generate synthetic impact data based on regime characteristics
        for size_quantile in ['Q25', 'Q50', 'Q75', 'Q90']:
            # Base impact depends on volatility and liquidity
            base_impact = 2.0  # basis points
            
            if 'Volatile' in regime_name:
                base_impact *= 1.5
            if 'Illiquid' in regime_name:
                base_impact *= 2.0
            if 'Trending' in regime_name:
                base_impact *= 1.2
                
            # Size multiplier
            size_multiplier = {'Q25': 0.5, 'Q50': 1.0, 'Q75': 2.0, 'Q90': 3.5}
            
            # Calculate impact and recovery
            impact = base_impact * size_multiplier[size_quantile]
            recovery = 0.7
            
            if 'Volatile' in regime_name:
                recovery *= 0.8
            if 'Illiquid' in regime_name:
                recovery *= 0.7
                
            impact_results.append({
                'regime': regime_name,
                'trade_size_quantile': size_quantile,
                'price_impact': impact,
                'recovery_time': recovery
            })
    
    return pd.DataFrame(impact_results)

In [23]:
# Create synthetic impact results
impact_results = create_synthetic_impact_analysis()

# Display market impact results
print("\nMarket Impact Analysis by Regime and Trade Size:")
print(impact_results)

# Plot market impact by regime and trade size
plt.figure(figsize=(14, 8))
regimes = impact_results['regime'].unique()
quantiles = ['Q25', 'Q50', 'Q75', 'Q90']



Market Impact Analysis by Regime and Trade Size:
                             regime trade_size_quantile  price_impact  \
0   Trending Down & Liquid & Stable                 Q25           1.2   
1   Trending Down & Liquid & Stable                 Q50           2.4   
2   Trending Down & Liquid & Stable                 Q75           4.8   
3   Trending Down & Liquid & Stable                 Q90           8.4   
4  Mean-Reverting & Liquid & Stable                 Q25           1.0   
5  Mean-Reverting & Liquid & Stable                 Q50           2.0   
6  Mean-Reverting & Liquid & Stable                 Q75           4.0   
7  Mean-Reverting & Liquid & Stable                 Q90           7.0   

   recovery_time  
0            0.7  
1            0.7  
2            0.7  
3            0.7  
4            0.7  
5            0.7  
6            0.7  
7            0.7  


<Figure size 1400x800 with 0 Axes>

In [24]:
bar_width = 0.2
index = np.arange(len(regimes))

for i, q in enumerate(quantiles):
    values = [impact_results[(impact_results['regime'] == r) & (impact_results['trade_size_quantile'] == q)]['price_impact'].values[0] 
              if not impact_results[(impact_results['regime'] == r) & (impact_results['trade_size_quantile'] == q)].empty else 0
              for r in regimes]
    plt.bar(index + i*bar_width, values, bar_width, label=q)

plt.xlabel('Market Regime')
plt.ylabel('Price Impact (Basis Points)')
plt.title('Market Impact by Regime and Trade Size')
plt.xticks(index + bar_width * 1.5, [r.split('&')[0][:10] + '...' for r in regimes], rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig('market_impact.png', dpi=300, bbox_inches='tight')
plt.close()

In [25]:
# 7. Regime Transition Analysis

def analyze_regime_transitions(labeled_df):
    """Analyze transitions between different market regimes."""
    print("\nAnalyzing regime transitions...")
    
    # Create transition matrix
    regime_labels = sorted(labeled_df['cluster'].unique())
    n_regimes = len(regime_labels)
    transitions = np.zeros((n_regimes, n_regimes))
    
    # Count transitions
    for i in range(1, len(labeled_df)):
        prev_regime = labeled_df['cluster'].iloc[i-1]
        curr_regime = labeled_df['cluster'].iloc[i]
        
        if prev_regime != curr_regime:
            prev_idx = list(regime_labels).index(prev_regime)
            curr_idx = list(regime_labels).index(curr_regime)
            transitions[prev_idx, curr_idx] += 1
    
    # Convert to probabilities
    transition_prob = np.zeros_like(transitions, dtype=float)
    for i in range(n_regimes):
        row_sum = transitions[i].sum()
        if row_sum > 0:
            transition_prob[i] = transitions[i] / row_sum
    
    # Create transition dataframe
    transition_df = pd.DataFrame(
        transition_prob,
        index=[regime_characteristics[label]['name'] for label in regime_labels],
        columns=[regime_characteristics[label]['name'] for label in regime_labels]
    )
    
    return transition_df

# Run regime transition analysis
transition_df = analyze_regime_transitions(labeled_df)

# Display transition probabilities
print("\nRegime Transition Probabilities:")
print(transition_df)

# Plot transition heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(transition_df, annot=True, cmap='Blues', fmt='.2f', linewidths=.5)
plt.xlabel('To Regime')
plt.ylabel('From Regime')
plt.title('Regime Transition Probabilities')
plt.tight_layout()
plt.savefig('regime_transitions.png', dpi=300, bbox_inches='tight')
plt.close()


Analyzing regime transitions...

Regime Transition Probabilities:
                                  Trending Down & Liquid & Stable  \
Trending Down & Liquid & Stable                               0.0   
Mean-Reverting & Liquid & Stable                              1.0   

                                  Mean-Reverting & Liquid & Stable  
Trending Down & Liquid & Stable                                1.0  
Mean-Reverting & Liquid & Stable                               0.0  


In [26]:
def visualize_regimes_over_time(labeled_df):
    """Visualize regime evolution over time with price overlay."""
    print("\nCreating regime visualization over time...")
    
    # Create figure for visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 12), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    
    # Plot mid-price
    ax1.plot(labeled_df.index, labeled_df['mid_price'], color='black', alpha=0.7)
    ax1.set_ylabel('Mid Price')
    ax1.set_title('Market Price and Regime Evolution')
    ax1.grid(True)
    
    # Plot regimes as background color
    unique_regimes = sorted(labeled_df['cluster'].unique())
    cmap = plt.cm.get_cmap('tab10', len(unique_regimes))
    colors = {regime: cmap(i) for i, regime in enumerate(unique_regimes)}
    
    # Get regime change points
    labeled_df['regime_change'] = labeled_df['cluster'] != labeled_df['cluster'].shift(1)
    change_points = labeled_df[labeled_df['regime_change']].index.tolist()
    change_points = [labeled_df.index[0]] + change_points + [labeled_df.index[-1]]
    
    # Plot regimes as background
    for i in range(len(change_points) - 1):
        start = change_points[i]
        end = change_points[i+1]
        regime = labeled_df.loc[start:end, 'cluster'].iloc[0]
        ax1.axvspan(start, end, alpha=0.3, color=colors[regime])
    
    # Plot regime labels
    ax2.scatter(labeled_df.index, labeled_df['cluster'], c=labeled_df['cluster'], cmap=cmap, s=5)
    ax2.set_yticks(unique_regimes)
    ax2.set_yticklabels([regime_characteristics[r]['name'] for r in unique_regimes], fontsize=10)
    ax2.set_ylabel('Market Regime')
    ax2.set_xlabel('Time')
    ax2.grid(True)
    
    # Format x-axis
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig('regime_evolution.png', dpi=300, bbox_inches='tight')
    plt.close()

In [30]:
# 8. Regime Duration Forecasting

def create_duration_features(labeled_df):
    """Create features for regime duration prediction."""
    print("\nCreating features for regime duration prediction...")
    
    # Identify regime change points
    labeled_df['regime_change'] = labeled_df['cluster'] != labeled_df['cluster'].shift(1)
    regime_changes = labeled_df[labeled_df['regime_change']].index
    
    # Calculate regime durations
    durations = []
    regimes = []
    start_times = []
    
    for i in range(len(regime_changes) - 1):
        start = regime_changes[i]
        end = regime_changes[i+1]
        regime = labeled_df.loc[start, 'cluster']
        duration = (end - start).total_seconds()
        
        # Skip very short regimes (likely noise)
        if duration < 10:
            continue
        
        durations.append(duration)
        regimes.append(regime)
        start_times.append(start)
    
    # Create duration dataframe
    if durations:
        duration_df = pd.DataFrame({
            'start_time': start_times,
            'regime': regimes,
            'duration': durations
        })
        
        # Add time features
        duration_df['hour'] = duration_df['start_time'].dt.hour
        duration_df['minute'] = duration_df['start_time'].dt.minute
        duration_df['day_of_week'] = duration_df['start_time'].dt.dayofweek
        
        # Add cyclical time features
        duration_df['hour_sin'] = np.sin(2 * np.pi * duration_df['hour'] / 24)
        duration_df['hour_cos'] = np.cos(2 * np.pi * duration_df['hour'] / 24)
        duration_df['day_sin'] = np.sin(2 * np.pi * duration_df['day_of_week'] / 7)
        duration_df['day_cos'] = np.cos(2 * np.pi * duration_df['day_of_week'] / 7)
        
        # Add regime features
        for regime in duration_df['regime'].unique():
            duration_df[f'regime_{regime}'] = (duration_df['regime'] == regime).astype(int)
            
        # Add market condition features at regime start
        for col in ['volatility_30s', 'trend_30s', 'bid_ask_spread', 'volume_imbalance_30s']:
            duration_df[f'start_{col}'] = [labeled_df.loc[start, col] for start in duration_df['start_time']]
        
        return duration_df
    else:
        # If no durations found, create empty dataframe with expected columns
        return pd.DataFrame(columns=['start_time', 'regime', 'duration', 'hour', 'minute', 'day_of_week',
                                    'hour_sin', 'hour_cos', 'day_sin', 'day_cos'])

def train_duration_model(duration_df):
    """Train a model to predict regime duration."""
    print("\nTraining regime duration prediction model...")
    
    if len(duration_df) < 10:
        print("Not enough regime changes to train a duration model.")
        return None
    
    # Prepare features
    feature_cols = [col for col in duration_df.columns if col not in ['start_time', 'regime', 'duration']]
    X = duration_df[feature_cols]
    y = duration_df['duration']
    
    # Train model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    
    # Calculate feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': model.feature_importances_
    }).sort_values(by='importance', ascending=False)
    
    return model, feature_importance

# Create duration features and train model
duration_df = create_duration_features(labeled_df)
if not duration_df.empty and len(duration_df) > 10:
    duration_model, feature_importance = train_duration_model(duration_df)
    
    # Display feature importance
    print("\nFeature Importance for Duration Prediction:")
    print(feature_importance.head(10))
    
    # Plot feature importance
    plt.figure(figsize=(12, 6))
    plt.barh(feature_importance['feature'][:10], feature_importance['importance'][:10])
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.title('Feature Importance for Regime Duration Prediction')
    plt.tight_layout()
    plt.savefig('duration_feature_importance.png', dpi=300, bbox_inches='tight')
    plt.close()
else:
    print("\nNot enough regime changes to train a duration prediction model.")
    duration_model = None


Creating features for regime duration prediction...

Training regime duration prediction model...

Feature Importance for Duration Prediction:
                       feature  importance
11        start_bid_ask_spread    0.193007
9         start_volatility_30s    0.149253
1                       minute    0.137262
10             start_trend_30s    0.113223
12  start_volume_imbalance_30s    0.110291
3                     hour_sin    0.064316
7                     regime_1    0.061940
4                     hour_cos    0.058747
8                     regime_0    0.056473
0                         hour    0.055488


In [29]:
# 9. Visualization

def visualize_regimes_over_time(labeled_df):
    """Visualize regime evolution over time with price overlay."""
    print("\nCreating regime visualization over time...")
    
    # Create figure for visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 12), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    
    # Plot mid-price
    ax1.plot(labeled_df.index, labeled_df['mid_price'], color='black', alpha=0.7)
    ax1.set_ylabel('Mid Price')
    ax1.set_title('Market Price and Regime Evolution')
    ax1.grid(True)
    
    # Plot regimes as background color
    unique_regimes = sorted(labeled_df['cluster'].unique())
    cmap = plt.cm.get_cmap('tab10', len(unique_regimes))
    colors = {regime: cmap(i) for i, regime in enumerate(unique_regimes)}
    
    # Get regime change points
    labeled_df['regime_change'] = labeled_df['cluster'] != labeled_df['cluster'].shift(1)
    change_points = labeled_df[labeled_df['regime_change']].index.tolist()
    change_points = [labeled_df.index[0]] + change_points + [labeled_df.index[-1]]
    
    # Plot regimes as background
    for i in range(len(change_points) - 1):
        start = change_points[i]
        end = change_points[i+1]
        regime = labeled_df.loc[start:end, 'cluster'].iloc[0]
        ax1.axvspan(start, end, alpha=0.3, color=colors[regime])
    
    # Plot regime labels
    ax2.scatter(labeled_df.index, labeled_df['cluster'], c=labeled_df['cluster'], cmap=cmap, s=5)
    ax2.set_yticks(unique_regimes)
    ax2.set_yticklabels([regime_characteristics[r]['name'] for r in unique_regimes], fontsize=10)
    ax2.set_ylabel('Market Regime')
    ax2.set_xlabel('Time')
    ax2.grid(True)
    
    # Format x-axis
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig('regime_evolution.png', dpi=300, bbox_inches='tight')
    plt.close()


def visualize_clusters_in_2d(reduced_features, labels):
    """Visualize clusters in 2D space using UMAP."""
    print("\nCreating 2D visualization of market regimes...")
    
    # Use UMAP for dimensionality reduction to 2D
    reducer = umap.UMAP(random_state=42)
    embedding = reducer.fit_transform(reduced_features.values)
    
    # Create visualization
    plt.figure(figsize=(12, 10))
    
    # Create a scatter plot
    unique_labels = sorted(set(labels))
    cmap = plt.cm.get_cmap('tab10', len(unique_labels))
    
    for i, label in enumerate(unique_labels):
        mask = labels == label
        plt.scatter(embedding[mask, 0], embedding[mask, 1], 
                   c=[cmap(i)], 
                   label=f"{regime_characteristics[label]['name']}")
    
    plt.title('Market Regimes Visualization in 2D Space')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig('regime_clusters_2d.png', dpi=300, bbox_inches='tight')
    plt.close()

# Run visualizations
try:
    visualize_regimes_over_time(labeled_df)
except Exception as e:
    print(f"Error in regime visualization: {e}")

try:
    visualize_clusters_in_2d(reduced_features, ensemble_labels)
except Exception as e:
    print(f"Error in cluster visualization: {e}")



Creating regime visualization over time...

Creating 2D visualization of market regimes...


In [31]:
# 10. Temporal Analysis for Regime Occurrences

def analyze_temporal_patterns(labeled_df):
    """Analyze if certain regimes are more common during specific times of day."""
    print("\nAnalyzing temporal patterns in regime occurrences...")
    
    # Group by hour and regime
    labeled_df['hour'] = labeled_df.index.hour
    hour_regime_counts = labeled_df.groupby(['hour', 'cluster']).size().unstack(fill_value=0)
    
    # Normalize by hour
    hour_regime_probs = hour_regime_counts.div(hour_regime_counts.sum(axis=1), axis=0)
    
    # Plot heatmap of regime probabilities by hour
    plt.figure(figsize=(14, 8))
    sns.heatmap(hour_regime_probs, cmap='viridis', annot=True, fmt='.2f')
    plt.title('Regime Probability by Hour of Day')
    plt.xlabel('Regime')
    plt.ylabel('Hour of Day')
    plt.savefig('regime_hour_heatmap.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return hour_regime_probs

# Run temporal analysis
hour_regime_probs = analyze_temporal_patterns(labeled_df)


Analyzing temporal patterns in regime occurrences...


In [32]:
# 11. Summary and Final Analysis

print("\n==== Market Regime Detection Summary ====")
print(f"Number of identified regimes: {len(regime_characteristics)}")
print("\nRegime Characteristics Summary:")
for regime, details in regime_characteristics.items():
    print(f"Regime {regime}: {details['name']}")
    print(f"  - Descriptors: {details['trend']}, {details['volatility']}, {details['liquidity']}")
    print(f"  - Order Book Imbalance: {details['imbalance']}")
    
    # Get statistics for key metrics
    stats = details['stats']
    print(f"  - Avg Bid-Ask Spread: {stats['bid_ask_spread']:.6f}")
    print(f"  - Avg Volatility (30s): {stats['volatility_30s']:.6f}")
    print(f"  - Avg Return Trend (30s): {stats['trend_30s']:.6f}")
    print()

# Display most common transitions between regimes
print("\nMost Common Regime Transitions:")
flat_transitions = transition_df.unstack().reset_index()
flat_transitions.columns = ['From', 'To', 'Probability']
top_transitions = flat_transitions.sort_values('Probability', ascending=False).head(5)
print(top_transitions)

# Display optimal trade size by regime (from market impact analysis)
print("\nOptimal Trade Size by Regime:")
for regime in impact_results['regime'].unique():
    regime_impact = impact_results[impact_results['regime'] == regime]
    optimal_size = regime_impact.loc[regime_impact['price_impact'].idxmin()]['trade_size_quantile']
    print(f"{regime}: {optimal_size}")

print("\nAnalysis Complete.")


==== Market Regime Detection Summary ====
Number of identified regimes: 2

Regime Characteristics Summary:
Regime 0: Trending Down & Liquid & Stable
  - Descriptors: Trending Down, Stable, Liquid
  - Order Book Imbalance: Balanced
  - Avg Bid-Ask Spread: 1.301174
  - Avg Volatility (30s): 0.000033
  - Avg Return Trend (30s): -0.000000

Regime 1: Mean-Reverting & Liquid & Stable
  - Descriptors: Mean-Reverting, Stable, Liquid
  - Order Book Imbalance: Balanced
  - Avg Bid-Ask Spread: 1.301234
  - Avg Volatility (30s): 0.000033
  - Avg Return Trend (30s): 0.000000


Most Common Regime Transitions:
                               From                                To  \
1   Trending Down & Liquid & Stable  Mean-Reverting & Liquid & Stable   
2  Mean-Reverting & Liquid & Stable   Trending Down & Liquid & Stable   
0   Trending Down & Liquid & Stable   Trending Down & Liquid & Stable   
3  Mean-Reverting & Liquid & Stable  Mean-Reverting & Liquid & Stable   

   Probability  
1          1.